In [229]:
import numpy as np
import pandas as pd
import os, progressbar, re, sys, time

from Bio import Seq
from Bio import AlignIO, SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import generic_protein
from Bio.Align import MultipleSeqAlignment

In [15]:
res = []
for f in os.listdir('./RNABindR_raw_sources/'):
    if "predictionsFor" in f:
        spec = " ".join(f.split('.txt')[0].split('_')[-2:])
        o = open(os.path.join('.', 'RNABindR_raw_sources', f), 'r')
        line = o.readline()
        if not 'XP' in line:
            ID = line.split('>')[1].split('|')[0][:4]
        else:
            ID = line.split('>')[1].split('|')[0]
        res.append([f, spec, ID])

In [17]:
df = pd.DataFrame(res, columns=['path', 'specie', 'ID'])

In [19]:
search_results = pd.read_csv('../Data/01_Oskar_identification/oskar_tracker_results/search_results.csv')

In [112]:
lost_seq = {
    "ADM0": "Messor pergandei",
    "JXPJ010": "Drosophila lebanonensis",
    "XP_0113": "Cerapachys biroi",
    "PRJN": "Atrachya menetriesi"
}

lost_seq_meta = {
    "Messor pergandei" : {"family": "Formicidae", "order": "Hymenoptera"},
    "Drosophila lebanonensis": {"family": "Drosophilidae", "order": "Diptera"},
    "Cerapachys biroi": {"family": "Formicidae", "order": "Hymenoptera"},
    "Atrachya menetriesi": {"family": "Chrysomelidae", "order": "Coleoptera"}
}

In [119]:
def match_ID_spec(x):
    res = search_results[search_results['id'].str.match(x)]
    if len(res) > 0:
        return res['species'].values[0]
    else:
        return None
    
def match_spec_spec(x):
    res = search_results[search_results['species'].str.lower().str.match(x)]
    if len(res) > 0:
        return res['species'].values[0]
    else:
        return None
    
def merge(y):
    if y['matched_ID_spec'] and y['matched_spec_spec']:
        if y['matched_ID_spec'] == y['matched_spec_spec']:
            return y['matched_ID_spec']
        else:
            print(y['matched_ID_spec'], y['matched_spec_spec'])
    elif y['matched_ID_spec'] or y['matched_spec_spec']:
        if y['matched_ID_spec']:
            return y['matched_ID_spec']
        else:
            return y['matched_spec_spec']
    else:
        return lost_seq[y['ID']]

def family(x):
    r = search_results[search_results['species'] == x]['family_name']
    if len(r) > 0:
        return r.values[0]
    else:
        return lost_seq_meta[x]['family']    
    
def order(x):
    r = search_results[search_results['species'] == x]['order_name']
    if len(r) > 0:
        return r.values[0]
    else:
        return lost_seq_meta[x]['order']    

In [114]:
df['matched_ID_spec'] = df['ID'].apply(match_ID_spec)

In [115]:
df['matched_spec_spec'] = df['specie'].apply(match_spec_spec)

In [116]:
df['species'] = df.apply(merge, axis=1)

In [118]:
df['family_name'] = df['species'].apply(family)

In [120]:
df['order_name'] = df['species'].apply(order)

In [122]:
del[df['specie']]
del[df['ID']]
del[df['matched_ID_spec']]
del[df['matched_spec_spec']]

In [124]:
df.to_csv('./RNABindR_raw_sources/RNA_binding_meta.csv', index=False)

## Remapping Dimeric Monomeric 

In [126]:
mapping = {
        'Gryllidae':"monomeric",
        'Formicidae':"monomeric",
        'Culicidae':"monomeric",
        'Pteromalidae':"dimeric",
        'Drosophilidae':"dimeric",
        'Tephritidae':"dimeric"
}

def mapdimer(x):
    if x in mapping:
        return mapping[x]
    else:
        return None

In [127]:
df['dimer_state'] = df['family_name'].apply(mapdimer)

In [130]:
dimer_sate_df = df[~df['dimer_state'].isnull()]

In [220]:
# Filter out duplicate sequences
tmp = dimer_sate_df.groupby('species', as_index=False).count()
duplicated = tmp[tmp["path"] > 1]['species'].values

In [217]:
seqlen = []
for path, specie, dimer in dimer_sate_df[['path', 'species', 'dimer_state']].values:
    f = open(os.path.join('.','RNABindR_raw_sources', path))
    lines = f.readlines()
    sequence = lines[1].split('sequence:')[1].strip()
    seqlen.append(len(sequence))
dimer_sate_df['seqlen'] = seqlen

/home/lblondel/anaconda3/envs/datascience/lib/python3.5/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [224]:
idx = dimer_sate_df.groupby(['species'])['seqlen'].transform(max) == dimer_sate_df['seqlen']

In [191]:
!mkdir tmp 

In [239]:
seqs = []
for path, specie, dimer in dimer_sate_df[idx][['path', 'species', 'dimer_state']].values:
    f = open(os.path.join('.','RNABindR_raw_sources', path))
    lines = f.readlines()
    sequence = lines[1].split('sequence:')[1].strip()
    seq = SeqIO.SeqRecord(Seq.Seq(sequence), id="RNABindR_{}".format(specie), name="RNABindR_{}".format(specie), description=specie)
    seqs.append(seq)
SeqIO.write(seqs, './tmp/RNABindR_sequences.fasta', 'fasta')

70

In [240]:
!mafft --add ./tmp/RNABindR_sequences.fasta --keeplength --reorder ../Data/01_Oskar_identification/oskar_tracker_results/oskar_filtered.aligned.fasta > ./tmp/RNABindR_sequences_aligned.fasta


nadd = 70
nthread = 0
stacksize: 8192 kb
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..

There are 1561 ambiguous characters.
  401 / 449
done.

Constructing a UPGMA tree (efffree=0) ... 
  440 / 449
done.

Progressive alignment 1/2... 
STEP   144 / 448 f
done.

Making a distance matrix from msa.. 
  400 / 449
done.

Constructing a UPGMA tree (efffree=1) ... 
  440 / 449
done.

Progressive alignment 2/2... 
STEP   147 / 448 f
done.

disttbfast (aa) Version 7.310 alg=A, model=BLOSUM62, 1.53, -0.00, -0.00, noshift, amax=0.0
0 thread(s)


To keep the alignment length, 96 letters were DELETED.
To know the positions of deleted letters, rerun the same command with the --mapout option.

Strategy:
 FFT-NS-2 (Fast but rough)
 Progressive method (guide trees were built 2 times.)

If unsure which option to use, try 'mafft --auto input > output'.
For more information, see 'mafft --help', 'mafft --man' and the mafft page.

The default gap scoring scheme has been changed in versio

In [241]:
align_path = "./tmp/RNABindR_sequences_aligned.fasta"
align_handle = SeqIO.parse(align_path, 'fasta')
alignment = [s for s in align_handle if 'RNABindR' in s.id]

In [244]:
species = {
    'dimeric': [],
    'monomeric': []
}
sequences = {
    'dimeric': [],
    'monomeric': []
}
scores = {
    'dimeric': [],
    'monomeric': []
}
for path, specie, dimer in dimer_sate_df[idx][['path', 'species', 'dimer_state']].values:
    f = open(os.path.join('.','RNABindR_raw_sources', path))
    lines = f.readlines()
    sequence = lines[1].split('sequence:')[1].strip()
    score = [float(i) for i in lines[3].split("predicted score:")[1].strip().split(',')]
    for seq in alignment:
        if specie in seq.description:
            break
    aligned_sequence = []
    aligned_score = []
    pos = 0
    for i in range(len(seq.seq)):
        if pos < len(sequence) and seq.seq[i] == sequence[pos]:
            aligned_sequence.append(sequence[pos])
            aligned_score.append(score[pos])
            pos += 1
        else:
            aligned_sequence.append('-')
            aligned_score.append(np.nan)
    if "".join(aligned_sequence) == seq.seq:
        species[dimer].append(specie)
        sequences[dimer].append(aligned_sequence)
        scores[dimer].append(aligned_score)

In [274]:
dimeric_score = np.nanmean(np.array(scores['dimeric']), axis=0)
monomeric_score = np.nanmean(np.array(scores['monomeric']), axis=0)

KeyError: 'dimeric'

In [260]:
scores = pd.read_csv('../Data/03_Oskar_scores_generation/CSV/scores.csv')
scores['RNABindR_dimeric'] = dimeric_score
scores['RNABindR_monomeric'] = monomeric_score
scores.to_csv('../Data/03_Oskar_scores_generation/CSV/scores.csv', index=False)

# Same for Holo Hemi

In [262]:
PGC_mode = {
    'Collembola':'Induction',
    'Diplura':'Induction',
    'Archaeognatha':'Induction',
    'Zygentoma':'Induction',
    'Odonata':'Induction',
    'Ephemeroptera':'Induction',
    'Zoraptera':'Induction',
    'Dermaptera':'Induction',
    'Plecoptera':'Induction',
    'Orthoptera':'Induction',
    'Mantophasmatodea':'Induction',
    'Grylloblattodea':'Induction',
    'Embioptera':'Induction',
    'Phasmatodea':'Induction',
    'Mantodea':'Induction',
    'Blattodea':'Induction',
    'Isoptera':'Induction',
    'Thysanoptera':'Induction',
    'Hemiptera':'Induction',
    'Psocoptera':'Induction',
    'Phthiraptera': 'Induction',
    'Hymenoptera':'Inheritance',
    'Raphidioptera':'Inheritance',
    'Megaloptera':'Inheritance',
    'Neuroptera':'Inheritance',
    'Strepsiptera':'Inheritance',
    'Coleoptera':'Inheritance',
    'Trichoptera':'Inheritance',
    'Lepidoptera':'Inheritance',
    'Siphonaptera':'Inheritance',
    'Mecoptera':'Inheritance',
    'Diptera':'Inheritance'
}

def set_germ_cell_formation(x):
    return PGC_mode[x]

In [263]:
df['pgc_state'] = df['order_name'].apply(set_germ_cell_formation)

In [264]:
pgc_state_df = df[~df['pgc_state'].isnull()]

In [265]:
# Filter out duplicate sequences
tmp = pgc_state_df.groupby('species', as_index=False).count()
duplicated = tmp[tmp["path"] > 1]['species'].values

In [266]:
seqlen = []
for path, specie in pgc_state_df[['path', 'species']].values:
    f = open(os.path.join('.','RNABindR_raw_sources', path))
    lines = f.readlines()
    sequence = lines[1].split('sequence:')[1].strip()
    seqlen.append(len(sequence))
pgc_state_df['seqlen'] = seqlen

In [267]:
idx = pgc_state_df.groupby(['species'])['seqlen'].transform(max) == pgc_state_df['seqlen']

In [191]:
!mkdir tmp 

In [269]:
seqs = []
for path, specie in pgc_state_df[idx][['path', 'species']].values:
    f = open(os.path.join('.','RNABindR_raw_sources', path))
    lines = f.readlines()
    sequence = lines[1].split('sequence:')[1].strip()
    seq = SeqIO.SeqRecord(Seq.Seq(sequence), id="RNABindR_{}".format(specie), name="RNABindR_{}".format(specie), description=specie)
    seqs.append(seq)
SeqIO.write(seqs, './tmp/RNABindR_sequences.fasta', 'fasta')

181

In [270]:
!mafft --add ./tmp/RNABindR_sequences.fasta --keeplength --reorder ../Data/01_Oskar_identification/oskar_tracker_results/oskar_filtered.aligned.fasta > ./tmp/RNABindR_sequences_aligned.fasta


nadd = 181
nthread = 0
stacksize: 8192 kb
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..

There are 1568 ambiguous characters.
  501 / 560
done.

Constructing a UPGMA tree (efffree=0) ... 
  550 / 560
done.

Progressive alignment 1/2... 
STEP   523 / 559 f
done.

Making a distance matrix from msa.. 
  500 / 560
done.

Constructing a UPGMA tree (efffree=1) ... 
  550 / 560
done.

Progressive alignment 2/2... 
STEP   511 / 559 f
done.

disttbfast (aa) Version 7.310 alg=A, model=BLOSUM62, 1.53, -0.00, -0.00, noshift, amax=0.0
0 thread(s)


To keep the alignment length, 233 letters were DELETED.
To know the positions of deleted letters, rerun the same command with the --mapout option.

Strategy:
 FFT-NS-2 (Fast but rough)
 Progressive method (guide trees were built 2 times.)

If unsure which option to use, try 'mafft --auto input > output'.
For more information, see 'mafft --help', 'mafft --man' and the mafft page.

The default gap scoring scheme has been changed in vers

In [271]:
align_path = "./tmp/RNABindR_sequences_aligned.fasta"
align_handle = SeqIO.parse(align_path, 'fasta')
alignment = [s for s in align_handle if 'RNABindR' in s.id]

In [272]:
species = {
    'Induction': [],
    'Inheritance': []
}
sequences = {
    'Induction': [],
    'Inheritance': []
}
scores = {
    'Induction': [],
    'Inheritance': []
}
for path, specie, dimer in pgc_state_df[idx][['path', 'species', 'pgc_state']].values:
    f = open(os.path.join('.','RNABindR_raw_sources', path))
    lines = f.readlines()
    sequence = lines[1].split('sequence:')[1].strip()
    score = [float(i) for i in lines[3].split("predicted score:")[1].strip().split(',')]
    for seq in alignment:
        if specie in seq.description:
            break
    aligned_sequence = []
    aligned_score = []
    pos = 0
    for i in range(len(seq.seq)):
        if pos < len(sequence) and seq.seq[i] == sequence[pos]:
            aligned_sequence.append(sequence[pos])
            aligned_score.append(score[pos])
            pos += 1
        else:
            aligned_sequence.append('-')
            aligned_score.append(np.nan)
    if "".join(aligned_sequence) == seq.seq:
        species[dimer].append(specie)
        sequences[dimer].append(aligned_sequence)
        scores[dimer].append(aligned_score)
    else:
        print(specie)

Palaephatus luteolus
Philopotamus ludificatus
Corethrella appendiculata
Drosophila albomicans
Drosophila serrata
Cryptocercus wrighti
Triarthria setipennis
Solenopsis invicta
Ceratitis capitata
Aedes aegypti
Drosophila simulans
Euwallacea fornicatus
Rhynchophorus ferrugineus
Anopheles quadriannulatus
Telenomus podisi
Atrachya menetriesi
Culex tarsalis


In [273]:
induction_score = np.nanmean(np.array(scores['Induction']), axis=0)
inheritance_score = np.nanmean(np.array(scores['Inheritance']), axis=0)

/home/lblondel/anaconda3/envs/datascience/lib/python3.5/site-packages/numpy/lib/nanfunctions.py:703: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)
/home/lblondel/anaconda3/envs/datascience/lib/python3.5/site-packages/numpy/lib/nanfunctions.py:703: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


In [260]:
scores = pd.read_csv('../Data/03_Oskar_scores_generation/CSV/scores.csv')
scores['RNABindR_h'] = induction_score
scores['RNABindR_monomeric'] = inheritance_score
scores.to_csv('../Data/03_Oskar_scores_generation/CSV/scores.csv', index=False)